In [1]:
import pandas as pd
import numpy as np
import gc; gc.enable()
import lightgbm as lgb
import warnings 
warnings.filterwarnings('ignore')

from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions

In [2]:
DATA_TYPE = 'sparse'
DATA_NORMALIZE = True
IS_REGRESSION = True
SEED = 519

In [3]:
dl = data_loader(data_type=DATA_TYPE, is_regression=IS_REGRESSION, is_train=True, is_pure=True)
train_X, train_y = dl.load()

Arguments: sparse True True
target loaded


In [4]:
dl_test = data_loader(data_type=DATA_TYPE, is_regression=IS_REGRESSION, is_train=False, is_pure=True)
test_X = dl_test.load()

Arguments: sparse True False
target loaded


In [5]:
train_X.shape, train_y.shape, test_X.shape

((1503424, 38406), (1503424,), (508438, 38406))

In [6]:
train_y

array([ 0.    ,  0.    ,  0.    , ...,  0.7376,  0.    ,  0.7376])

## KNN in PySparNN

In [7]:
import pysparnn.cluster_index as ci

In [8]:
k = 50
batch_size = 1
DEBUG = False

In [9]:
def get_predictions(tr_x, te_X):
    num_loops = int(np.ceil(te_X.shape[0]/batch_size))

    cp = ci.MultiClusterIndex(tr_X, range(tr_X.shape[0]))
    print('cp building done.\n')
    k_xvals, k_indices = [], []
    
    for i in tqdm(range(num_loops)):
        min_index = i*batch_size
        max_index = min((i+1)*batch_size, te_X.shape[0])
        x_batch = te_X[min_index:max_index, :]
        
        k_xval, k_indice = cp.search(te_X, k=k, return_distance=True)
        k_xvals.extend(k_xval)
        k_indices.extend(k_indice)
        
        if DEBUG:
            print(k_xvals, k_indice, k_xvals, k_indices)
           
        
        del k_xval, k_indice; gc.collect()
        
    return k_xvals, k_indices

In [10]:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=SEED)

fold=1
for train_ix, val_ix in kf.split(train_X):
    print('Processing fold # =', fold)
    
    tr_X, tr_y = train_X[train_ix,:], train_y[train_ix]
    val_X = train_X[val_ix,:]
    
    val_dist, val_ind = get_predictions(tr_X, val_X)
    test_dist, test_ind = get_predictions(tr_X, test_X)
    
    with open('knn_dist_and_ind_fold_{}.data'.format(fold), 'wb') as handle:
        pickle.dump(
            {
                'train_ix': train_ix,
                'val_dist': val_dist, 
                'val_ind': val_ind,
                'test_dist': test_dist,
                'test_ind': test_ind
            }, handle)
    
    del val_dist, val_ind, test_dist, test_ind; gc.collect()
    fold += 1

Processing fold # = 1


KeyboardInterrupt: 